# Quasi-dynamic spring slider with 1 state variable

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import time

import plotly.graph_objects as go

## Change Scipy's stepper method

In [ ]:
import scipy.integrate._ivp.rk as rk

In [ ]:
lyap = []
ndim = 2
def _step_impl_lyap(self):
        t = self.t
        y = self.y

        max_step = self.max_step
        rtol = self.rtol
        atol = self.atol

        min_step = 10 * np.abs(np.nextafter(t, self.direction * np.inf) - t)

        if self.h_abs > max_step:
            h_abs = max_step
        elif self.h_abs < min_step:
            h_abs = min_step
        else:
            h_abs = self.h_abs

        step_accepted = False
        step_rejected = False

        while not step_accepted:
            if h_abs < min_step:
                return False, self.TOO_SMALL_STEP

            h = h_abs * self.direction
            t_new = t + h

            if self.direction * (t_new - self.t_bound) > 0:
                t_new = self.t_bound

            h = t_new - t
            h_abs = np.abs(h)

            y_new, f_new = rk.rk_step(self.fun, t, y, self.f, h, self.A,
                                   self.B, self.C, self.K)
            scale = atol + np.maximum(np.abs(y), np.abs(y_new)) * rtol
            error_norm = self._estimate_error_norm(self.K, h, scale)

            if error_norm < 1:
                if error_norm == 0:
                    factor = rk.MAX_FACTOR
                else:
                    factor = min(rk.MAX_FACTOR,
                                 rk.SAFETY * error_norm ** self.error_exponent)

                if step_rejected:
                    factor = min(1, factor)

                h_abs *= factor

                step_accepted = True
                
                ## QR algorithm
                ## 1. local lyapunov exponents
                Mn = np.reshape(y_new[ndim:],(ndim,ndim)).transpose()
                Q,R = np.linalg.qr(Mn)
                lyap.append(np.log(abs(R.diagonal())))
                ## 2. change solution for the perturbation vectors
                y_new[ndim:] = Q.transpose().reshape(-1)
                
                
            else:
                h_abs *= max(rk.MIN_FACTOR,
                             rk.SAFETY * error_norm ** self.error_exponent)
                step_rejected = True
         
        self.h_previous = h
        self.y_old = y

        self.t = t_new
        self.y = y_new

        self.h_abs = h_abs
        self.f = f_new

        return True, None

In [ ]:
rk.RungeKutta._step_impl = _step_impl_lyap

### Parameters

In [ ]:
### Rate-and-state
# reference friction coefficient
fr = 0.6 
# reference slip velocity (m/s)
Vr = 1e-6
# direct effect parameter 
a = 0.015
#a = 0.020
# state evolution parameter
b = 0.018
# state evolution distance (m)
dc = 1e-2

### Elasticity
# density (g/cm^3)
rho = 2.54
# shear velocity (m/s)
c = 2.5
# radiation damping (MPa*s/m)
eta = rho*c/2
# normal stress (MPa)
sigma = 50
# initial shear stress (MPa)
tau0 = 0.5793*sigma
# calculate critical spring constant (MPa/m)
kcr = sigma*abs(b-a)/dc
# spring constante (MPa/m)
k = 2*kcr
# loading: shear stress increases at constant rate in 'absence' of slip
oneyear = np.pi*1e7
# delta tau (MPa/s)
dtau = 10/oneyear

#### calculate bifurcation values with radiation damping
B = sigma/dc * (a-b)
C = dtau*eta/dc

kcr1 = (-B+np.sqrt(B**2-4*C))/2
kcr2 = (-B-np.sqrt(B**2-4*C))/2
print(kcr1)
print(kcr2)

k = 0.2*kcr1

################


### Non-dimensional groups (combos of above)
eps1 = dtau/(k*Vr) 
eps2 = sigma*fr/(eta*Vr)
eps3 = a/fr
eps4 = eta*Vr/(k*dc*fr)


### Set up for solve

In [ ]:
### Time (s)
# max simulation time (s)
tmax = 10*oneyear
# initial time (s)
t0 = 0


### Initial conditions
# Initial slip velocity (m/s)
v0 = 1e-9
# Initial friciton coefficient
f0 = 0.57
# dimension-less versions
x0 = np.log(v0/Vr)
y0 = f0/fr
T0 = t0*k/eta
Tmax = tmax*k/eta
# stack initial values
s0 = np.zeros(2)
s0[0] = x0
s0[1] = y0
pertb0= np.eye(2).reshape(-1,1)
pertb0=pertb0[:,0]
stacked0 = np.concatenate((s0,pertb0))
print(stacked0)

t_span = np.array([T0, Tmax])
maxStep = 1e9*k/eta

In [ ]:
def jac(x,y):
    bigBracketX = eps1*np.exp(-x)-1-eps2*eps4*(fr*(1-y)+x*(a-b))
    bigBracketY = eps1*eps3*np.exp(-x)-eps3+np.exp(x)*eps4*(fr*(1-y)+x*(a-b))
    
    Xx = -(1+eps2*eps3*(np.exp(-x)))**(-2)*(-eps2*eps3*np.exp(-x))*bigBracketX + \
         (1+eps2*eps3*np.exp(-x))**(-1)*(-eps1*np.exp(-x)-eps2*eps4*(a-b))
        
    Xy = (1+eps2*eps3*np.exp(-x))**(-1) *eps2*eps4*fr
    
    Yx = -(1+eps2*eps3*(np.exp(-x)))**(-2)*(-eps2*eps3*np.exp(-x))*bigBracketY + \
         (1+eps2*eps3*np.exp(-x))**(-1)* \
         (-eps1*eps3*np.exp(-x)+np.exp(x)*eps4*(fr*(1-y)+x*(a-b))+np.exp(x)*eps4*(a-b))
            
    Yy = -(1+eps2*eps3*np.exp(-x))**(-1)*np.exp(x)*eps4*fr

    
    return np.array([[Xx, Xy],
                     [Yx, Yy]])

In [ ]:
def nonDimSliderSystem_stacked(T,x):
    # T: non-dimensional time
    # x:
    # x[0] = x = ln(v/Vr)
    # x[1] = y = f/fr

    
    # a common factor:
    cf = eps4*(f0*(1-x[1])+(a-b)*x[0])
    
    sysdot = np.array( [(eps1*np.exp(-x[0]) -1 - eps2*cf)/(1+eps2*eps3*np.exp(-x[0])),
         
                     (eps1*eps3*np.exp(-x[0])-eps3+np.exp(x[0])*cf)/(1+eps2*eps3*np.exp(-x[0])) ])
    
    pertb_vecs = x[2:].reshape((2,2)).transpose()
    Mdot = np.matmul(jac(x[0],x[1])[:,:,0], pertb_vecs).transpose().reshape(-1,1)
    
    return np.concatenate((sysdot,Mdot))
   
    

## Scipy integrate the stacked system

In [ ]:
start_time = time.time()
sol=solve_ivp(nonDimSliderSystem_stacked, t_span, stacked0, method='RK45', t_eval=None,
              dense_output=False, events=None, vectorized=True,
              args=None,max_step=maxStep,rtol=1e-10)
end_time = time.time()
print("--- Execution time is %s seconds ---" % (end_time - start_time))

In [ ]:
sol.t[-1]

## Calculate Lyapunov exponents

In [ ]:
local_lyap = np.array(lyap).transpose()
le = np.divide(np.cumsum(local_lyap,axis=1),sol.t[1:])

In [ ]:
plt.rcParams.update({'font.size': 16})
### Plot Lyapunov exponents
fig, ax = plt.subplots(nrows = 1, ncols = 1,figsize=(10,8))
ax.plot(sol.t[1:],le[0,:],color='blue')
ax.plot(sol.t[1:],le[1,:],color='cyan')

# plot the sections where x > 10
sidex = np.argwhere(sol.y[0]>=10)
ax.plot(sol.t[sidex+1],le[0,sidex+1],'ro',markersize=2)
ax.plot(sol.t[sidex+1],le[1,sidex+1],'ro',markersize=2)

ax.set(xlabel='Nondimensional time',ylabel='Lyapunov exponents')

## 